# China_city_target_GSP data preprocessing

This notebook has been generated on 2019-08-26 11:04 

The objective of this notebook is to YYY

## Global steps 

The global steps to construct the dataset are the following:

- Lorem ipsum dolor sit amet
- Lorem ipsum dolor sit amet
- Lorem ipsum dolor sit amet

## Data source 

The data source to construct the dataset are the following:


 ### Big Query Dataset 
 
 ### Google Cloud Storage Dataset 
 
 ### Google Spreadsheet Dataset 
 
 - [chinese_city_characteristics](https://docs.google.com/spreadsheets/1-x9DCX4cun6Ed9iH5MiI4g21fXm7seRHHz7WXGiVKVU) 
 - [China_cities_target_so2](https://docs.google.com/spreadsheets/1z3A_I8_StdyNL5O38s2l9hx6W3VR49CGVmaosypjFMA)

## Destination

The new dataset is available from XXX

- GS: None
- GCS: China_city_target_GSP.gz
- BG: China_city_target_GSP

# Load Dataset


In [ ]:
from Fast_connectCloud import connector
from GoogleDrivePy.google_drive import connect_drive
from GoogleDrivePy.google_platform import connect_cloud_platform
import pandas as pd 
import numpy as np
import pandas_profiling

In [ ]:
gs = connector.open_connection(online_connection = False, 
	path_credential = '/Users/Thomas/Google Drive/Projects/Data_science/Google_code_n_Oauth/Client_Oauth/Google_auth/')

service_gd = gs.connect_remote(engine = 'GS')
service_gcp = gs.connect_remote(engine = 'GCP')

gdr = connect_drive.connect_drive(service_gd['GoogleDrive'])

project = 'valid-pagoda-132423'
gcp = connect_cloud_platform.connect_console(project = project,
											 service_account = service_gcp['GoogleCloudP'])


## Load chinese_city_characteristics from Google Spreadsheet

Feel free to add description about the dataset or any usefull information.

Profiling will be available soon for this dataset

We removed columns AC:CP in GS because the function upload_data cannot loads data after columns AC

In [ ]:

### Please go here https://docs.google.com/spreadsheets/d/1-x9DCX4cun6Ed9iH5MiI4g21fXm7seRHHz7WXGiVKVU
### To change the range

sheetid = '1-x9DCX4cun6Ed9iH5MiI4g21fXm7seRHHz7WXGiVKVU'
sheetname = 'chinese_city_characteristics.csv'

df_chinese_city_characteristics = gdr.upload_data_from_spreadsheet(sheetID = sheetid,
sheetName = sheetname,
	 to_dataframe = True)
df_chinese_city_characteristics.head()



## Load China_cities_target_so2 from Google Spreadsheet

Feel free to add description about the dataset or any usefull information.

Profiling will be available soon for this dataset



In [ ]:

### Please go here https://docs.google.com/spreadsheets/d/1z3A_I8_StdyNL5O38s2l9hx6W3VR49CGVmaosypjFMA
### To change the range

sheetid = '1z3A_I8_StdyNL5O38s2l9hx6W3VR49CGVmaosypjFMA'
sheetname = 'China_cities_target_so2'

df_China_cities_target_so2 = gdr.upload_data_from_spreadsheet(sheetID = sheetid,
sheetName = sheetname,
	 to_dataframe = True)
df_China_cities_target_so2.head()


## Load cityname_and_code from Google Spreadsheet

Feel free to add description about the dataset or any usefull information.

Profiling will be available soon for this dataset

In [ ]:
### Please go here https://docs.google.com/spreadsheets/d/1fIziz-Xt99-Rj6NLm52-i6jScOLXgAY20KJi8k3DruA
### To change the range

sheetid = '1fIziz-Xt99-Rj6NLm52-i6jScOLXgAY20KJi8k3DruA'
sheetname = 'final'

df_cityname_and_code = gdr.upload_data_from_spreadsheet(sheetID = sheetid,
sheetName = sheetname,
	 to_dataframe = True)
df_cityname_and_code.head()

# Workflow

In this section, we will construct the dataset, and document each step of the workflow.

Please use the following format for the documentation:

- `##` Step 1: XXX
- `###` (optional) Underlying process description
- `##` Step 2: YYY
- `###` (optional) Underlying process description

Note: **You need to rename the last dataframe `df_final`**

## Step 1: Merge city chinese/english

In [ ]:
cities = (df_cityname_and_code
          .merge(df_China_cities_target_so2,
                          on = 'citycn', 
                          how = 'inner')
          .drop_duplicates(subset = 'citycn',
                           keep = 'first')
         )
cities['year'] = '2005'

In [ ]:
to_keep = ['geocode4_corr', 'year', 'gdp', 'population']
cities_gdp = cities.merge(df_chinese_city_characteristics[to_keep],
                on = ['geocode4_corr', 'year'])


In [ ]:
df_final = cities_gdp.apply(pd.to_numeric, errors='ignore')

In [ ]:
df_final['gdp_cap'] = df_final['gdp'] / df_final['population'] 

# Profiling

In order to get a quick summary statistic of the data, we generate an HTML file with the profiling of the dataset we've just created. 

The profiling will be available at this URL after you commit a push to GitHub. 

**You need to rename the final dataframe `df_final` in the previous section to generate the profiling.**

In [ ]:
#### make sure the final dataframe is stored as df_final
profile = pandas_profiling.ProfileReport(df_final)
name_html = "China_city_target_GSP.html"
profile.to_file(output_file=name_html)

# Upload to cloud

The dataset is ready to be shared with your colleagues. 





### Move to GCS and BigQuery

We move the dataset to the following:

- **bucket**: *NEED TO DEFINE*

- **Destination_blob**: *XXXXX/Processed_*
- **name**:  *China_city_target_GSP.gz*
- **Dataset**: *China*

- **table**: *China_city_target_GSP*

### GCS

We first need to save *China_city_target_GSP* with `.gz` extension locally then we can move it
to GCS


In [ ]:

### First save locally
df_final.to_csv(
	'China_city_target_GSP.gz',
	sep=',',
	header=True,
	index=False,
	chunksize=100000,
	compression='gzip',
	encoding='utf-8')

### Then upload to GCS
bucket_name = 'chinese_data'
destination_blob_name = 'Panel_china/Cities_panel_china/Processed_cities_china'
source_file_name = 'China_city_target_GSP.gz'
gcp.upload_blob(bucket_name, destination_blob_name, source_file_name)



In [ ]:


### Move to bigquery
bucket_gcs ='chinese_data/Panel_china/Cities_panel_china/Processed_cities_china/China_city_target_GSP.gz'
gcp.move_to_bq_autodetect(dataset_name= 'China',
							 name_table= 'China_city_target_GSP',
							 bucket_gcs=bucket_gcs)
